In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285398 sha256=3ddf8b4033d095c355c3d132b2ef182a52230a0ff50fa842c0fd5cdba9897b63
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [2]:
import pyspark
import re
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.functions import regexp_replace

In [3]:
spark = SparkSession.builder.getOrCreate()

In [4]:
options = {"header": "true","quote": "\"","escape": "\"","multiLine": "true"}
data=spark.read.csv(path="/content/data/details",**options).withColumn("Comments", regexp_replace("Comments", '\n', "")).withColumn("Comments",regexp_replace("Comments","^\"|\"$", ""))

In [5]:
data.show(truncate=False)

+-----+-----+-----+--------------------------------------------+------------------------------------------------------------------------+
|FNAME|LNAME|SSN  |ADDR                                        |Comments                                                                |
+-----+-----+-----+--------------------------------------------+------------------------------------------------------------------------+
|Ash  |B    |1234 |9425 Rolater Rd, Apt 832, Frisco, TX,  75068|This is a Text. This means "nothing","06/23/2023" is the data           |
|Jack |Jack |34567|9300 Coit Rd Apt 711 Plano Tx 75025         |This, is" also a, comment "but means, """Something""".                  |
|Sage |IT   |56789|8590 GaryBurns Dr, Frisco, Tx, 75034        |This is absolute, "???Bonkers???.",This is another line in the same row.|
+-----+-----+-----+--------------------------------------------+------------------------------------------------------------------------+



In [6]:
data.select("Comments").show(truncate=False)

+------------------------------------------------------------------------+
|Comments                                                                |
+------------------------------------------------------------------------+
|This is a Text. This means "nothing","06/23/2023" is the data           |
|This, is" also a, comment "but means, """Something""".                  |
|This is absolute, "???Bonkers???.",This is another line in the same row.|
+------------------------------------------------------------------------+



In [7]:
print(type(data))

<class 'pyspark.sql.dataframe.DataFrame'>


In [8]:
marks=[23,34,45,56,78]
new_marks= [i+2 for i in marks]

In [9]:
from pyspark.sql.functions import regexp_extract, col

In [10]:
data.createOrReplaceTempView("df1")

In [11]:
result = spark.sql("SELECT Comments from df1 where df1.Comments like '%nothing%'")

In [12]:
result.show(truncate=False)

+-------------------------------------------------------------+
|Comments                                                     |
+-------------------------------------------------------------+
|This is a Text. This means "nothing","06/23/2023" is the data|
+-------------------------------------------------------------+



In [13]:
result = data.select(regexp_extract("Comments", "(\\d{2}/\\d{2}/\\d{4})", 1).alias('date'))

In [14]:
result.show(truncate=False)

+----------+
|date      |
+----------+
|06/23/2023|
|          |
|          |
+----------+



In [15]:
from pyspark.sql.functions import regexp_replace, lower
df = data.withColumn("Comments", regexp_replace(data.Comments, "\\?", "!"))


In [16]:
df.show(truncate=False)

+-----+-----+-----+--------------------------------------------+------------------------------------------------------------------------+
|FNAME|LNAME|SSN  |ADDR                                        |Comments                                                                |
+-----+-----+-----+--------------------------------------------+------------------------------------------------------------------------+
|Ash  |B    |1234 |9425 Rolater Rd, Apt 832, Frisco, TX,  75068|This is a Text. This means "nothing","06/23/2023" is the data           |
|Jack |Jack |34567|9300 Coit Rd Apt 711 Plano Tx 75025         |This, is" also a, comment "but means, """Something""".                  |
|Sage |IT   |56789|8590 GaryBurns Dr, Frisco, Tx, 75034        |This is absolute, "!!!Bonkers!!!.",This is another line in the same row.|
+-----+-----+-----+--------------------------------------------+------------------------------------------------------------------------+



In [17]:
print(type(df))

<class 'pyspark.sql.dataframe.DataFrame'>


In [18]:
df.createOrReplaceTempView("data_frame")

In [19]:
lowercase_replace = udf(lambda Comments: re.sub('("(.*?)")', lambda match: match.group(0).lower(), Comments))
result1 = df.withColumn("Comments", lowercase_replace("Comments"))


In [20]:
print(type(result1))

<class 'pyspark.sql.dataframe.DataFrame'>


In [21]:
result1.show(truncate=False)

+-----+-----+-----+--------------------------------------------+------------------------------------------------------------------------+
|FNAME|LNAME|SSN  |ADDR                                        |Comments                                                                |
+-----+-----+-----+--------------------------------------------+------------------------------------------------------------------------+
|Ash  |B    |1234 |9425 Rolater Rd, Apt 832, Frisco, TX,  75068|This is a Text. This means "nothing","06/23/2023" is the data           |
|Jack |Jack |34567|9300 Coit Rd Apt 711 Plano Tx 75025         |This, is" also a, comment "but means, """something""".                  |
|Sage |IT   |56789|8590 GaryBurns Dr, Frisco, Tx, 75034        |This is absolute, "!!!bonkers!!!.",This is another line in the same row.|
+-----+-----+-----+--------------------------------------------+------------------------------------------------------------------------+



In [22]:
uppercase_replace = udf(lambda Comments: re.sub('("(.*?)")', lambda match: match.group(0).upper(), Comments))
result2 = df.withColumn("Comments", uppercase_replace("Comments"))


In [23]:
result2.show(truncate=False)

+-----+-----+-----+--------------------------------------------+------------------------------------------------------------------------+
|FNAME|LNAME|SSN  |ADDR                                        |Comments                                                                |
+-----+-----+-----+--------------------------------------------+------------------------------------------------------------------------+
|Ash  |B    |1234 |9425 Rolater Rd, Apt 832, Frisco, TX,  75068|This is a Text. This means "NOTHING","06/23/2023" is the data           |
|Jack |Jack |34567|9300 Coit Rd Apt 711 Plano Tx 75025         |This, is" ALSO A, COMMENT "but means, """SOMETHING""".                  |
|Sage |IT   |56789|8590 GaryBurns Dr, Frisco, Tx, 75034        |This is absolute, "!!!BONKERS!!!.",This is another line in the same row.|
+-----+-----+-----+--------------------------------------------+------------------------------------------------------------------------+

